In [1]:
from huggingface_hub import hf_hub_download
# downloaded_model_path = hf_hub_download(repo_id="bartowski/Llama-3.2-3B-Instruct-GGUF", filename="Llama-3.2-3B-Instruct-Q6_K_L.gguf") 
# # LLama 3.2 sehr schlechtes Ergebnis aber nur 20sec zum Generieren
downloaded_model_path = hf_hub_download(repo_id="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF", filename="Meta-Llama-3.1-8B-Instruct-Q6_K_L.gguf") 
# # LLama 3.1 sehr gutes Ergebnis, sehr dauert auch nur 20sec 
#downloaded_model_path = hf_hub_download(repo_id="bartowski/Mistral-Nemo-Instruct-2407-GGUF", filename="Mistral-Nemo-Instruct-2407-Q4_K_M.gguf") 
# 6_K_L: Mistral Nemo sehr gutes Ergebnis extrem langsam 3min, eventuell könnte 3 gut sein oder 4, Q4_K_M: sehr gutes Ergebnis, dauert ungefähr 1min
# downloaded_model_path = hf_hub_download(repo_id="TheBloke/Mistral-7B-Instruct-v0.2-GGUF", filename="mistral-7b-instruct-v0.2.Q5_K_M.gguf") 
# normal Mistral schlechtes Ergebnis, aber besser als llama 3.2 dauert so lange wie llama 3.2
#downloaded_model_path = hf_hub_download(repo_id="bartowski/Mistral-NeMo-Minitron-8B-Instruct-GGUF", filename="Mistral-NeMo-Minitron-8B-Instruct-Q4_K_L.gguf") 
# Q6_K_L Minitron sehr gutes Ergebnis, dauert eine Minute, Q5_K_L und Q4_K_L jeweils pro Q1 10sec schneller (Q5_K_M, Q4_K_M): buggy liefert nicht den richtigen Output
# Gemma -> not able to generate json output
#downloaded_model_path = hf_hub_download(repo_id="bartowski/gemma-2-9b-it-GGUF", filename="gemma-2-9b-it-Q4_K_L.gguf") 

# Modelle, die ich miteinander vergleichen will:
# LLama 3.1
# -> bestes Model, will ich finetunen und gegen das normale Model als Baseline vergleichen

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
"""

def get_prompt(message: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f'[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message} [/INST]'

In [2]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum

class Zustand(str, Enum):
    RENOVIERT = "Renoviert"
    NEU = "Neu"
    GEPFLEGT = "Gepflegt"
    
class Heizsystem(str, Enum):
    GAS = "Gas"
    STROM = "Strom"
    FERN = "Fernwärme"
    ÖL = "Öl"
    FESTBRENNSTOFFE = "Festbrennstoffe"

class RaumTyp(str, Enum):
    WOHNZIMMER = "Wohnzimmer"
    SCHLAFZIMMER = "Schlafzimmer"
    KUECHE = "Küche"
    BADEZIMMER = "Badezimmer"
    ARBEITSZIMMER = "Arbeitszimmer"
    HAUSWIRTSCHAFTSRAUM = "Hauswirtschaftsraum"
    FLUR = "Flur"
    ABSTELLRAUM = "Abstellraum"
    ESSZIMMER = "Esszimmer"

class VersteigerungsTyp(str, Enum):
    WOHNUNG = "Wohnung"
    HAUS = "Haus"
    ANDERES = "Anderes"

class ForclosureObject(BaseModel):
    flaeche: int = Field(description="Die Fläche des Objekts in Quadratmetern.")
    # Generiert zu viele 
    # beschreibung: str = Field(description="Beschreibung des Zwangsversteigerungsobjekts.")
    verkehrswert: int = Field(description="Verkehrswert des Objekts.")
    typ: Optional[VersteigerungsTyp] = Field(description="Art der Immobilie (z.B. Wohnung, Haus oder etwas anderes).")
    baujahr: Optional[int] = Field(description="Baujahr der Immobilie.")
    heizsystem: Optional[Heizsystem] = Field(description="Art des Heizsystems.")
    # NOTE: Der Zustand wird nicht beschrieben in den Bekanntmachungen, aber in den Exposees, diese sind aber zu lang
    # zustand: Optional[Zustand] = Field(description="Zustand des Objekts.")
    raeume: Optional[int] = Field(description="Anzahl der Räume im Objekt.")
    raum_typen: List[RaumTyp] = Field(default_factory=list, description="""Liste der Raumtypen im Objekt (z.B. Wohnzimmer, Küche).
                                      Die Anzahl der Einträge in der Liste sollte der Gesamtzahl der Räume entsprechen. 
                                      Mehrere Räume desselben Typs sollten jeweils einzeln aufgeführt werden.""")
    balkon: bool = Field(description="Gibt an, ob das Objekt einen Balkon hat.")
    garten: bool = Field(description="Hat das Objekt einen Garten.")
    # NOTE: Werde ich nicht in meinen predictions berücksichtigen und könnte eventuell die Extraktion beeinflussen
    #vermietet: bool = Field(description="Gibt an, ob das Objekt vermietet ist.")
    #miete: Optional[str] = Field(description="Monatliche Mieteinnahmen für das Objekt, falls vermietet.")

class Forclosure(BaseModel):
    objekte: List[ForclosureObject] = Field(description="Liste der Zwangsversteigerungsobjekte, die zu diesem Fall gehören.")
    gesamtverkehrswert: int = Field(description="Gesamtverkehrswert aller Zwangsversteigerungsobjekte.")
    #beschreibung: str = Field(description="Allgemeine Beschreibung des Zwangsversteigerungsfalls.")


In [3]:
from llama_cpp import Llama
llm = Llama(model_path=downloaded_model_path, n_ctx=16384, n_threads=8, n_gpu_layers=-1, verbose=False, temperature=0)

In [4]:
import pymupdf

def pdf_to_string(file_path):
    pdf_document = pymupdf.open(file_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text += page.get_text()
    pdf_document.close()
    return text

In [5]:
from typing import Optional
from llama_cpp import LogitsProcessorList
from lmformatenforcer import CharacterLevelParser
from lmformatenforcer.integrations.llamacpp import build_llamacpp_logits_processor
from lmformatenforcer import JsonSchemaParser
from IPython.display import display, Markdown

def display_header(text):
    display(Markdown(f'**{text}**'))

def display_content(text):
    display(Markdown(f'```\n{text}\n```'))

def llamacpp_with_character_level_parser(llm: Llama, prompt: str, character_level_parser: Optional[CharacterLevelParser]) -> str:
    logits_processors: Optional[LogitsProcessorList] = None
    if character_level_parser:
        logits_processors = LogitsProcessorList([build_llamacpp_logits_processor(llm, character_level_parser)])
    
    output = llm(prompt, logits_processor=logits_processors, max_tokens=1000)
    text: str = output['choices'][0]['text']
    return text

In [6]:
# for single files
# file_path = "../test_pdfs/359816.pdf"
# pdf_text = pdf_to_string(file_path)
# question_with_schema = f'You MUST answer using the following json schema: {Forclosure.model_json_schema()}. \n Please extract information about {pdf_text}.'
# prompt = get_prompt(question_with_schema)
# display_header("LLM Output with json schema enforcing:")
# result = llamacpp_with_character_level_parser(llm, prompt, JsonSchemaParser(Forclosure.model_json_schema()))
# display_content(result)


In [ ]:
import os
test_pdfs_dir = "../test_pdfs"
with open("./examples/5shot.txt", "r") as shot:
    examples = shot.read()

results = []

for file_name in os.listdir(test_pdfs_dir):
    file_path = os.path.join(test_pdfs_dir, file_name)
    
    if file_name.endswith(".pdf"):
        pdf_text = pdf_to_string(file_path)
        
        question_with_schema = (
            f'The following text lists examples for your task: {examples}.\n'
            f'You MUST answer using the following JSON schema: {Forclosure.model_json_schema()}.\n'
            f'Please extract information about the following PDF content: {pdf_text}.'
        )
        
        prompt = get_prompt(question_with_schema)
        display_header(f"LLM Output for {file_name} with JSON schema enforcing:")
        
        result = llamacpp_with_character_level_parser(
            llm, prompt, JsonSchemaParser(Forclosure.model_json_schema())
        )
        results.append(result)

**LLM Output for 105869.pdf with JSON schema enforcing:**

**LLM Output for 113772.pdf with JSON schema enforcing:**

**LLM Output for 127734.pdf with JSON schema enforcing:**

**LLM Output for 15125.pdf with JSON schema enforcing:**

**LLM Output for 27138.pdf with JSON schema enforcing:**

**LLM Output for 359816.pdf with JSON schema enforcing:**

**LLM Output for 60782.pdf with JSON schema enforcing:**

**LLM Output for 61344.pdf with JSON schema enforcing:**

**LLM Output for 8033.pdf with JSON schema enforcing:**

**LLM Output for 8786.pdf with JSON schema enforcing:**

In [11]:
output_file_path = "output.txt"
with open(output_file_path, "w", encoding="utf-8") as file:
    for i, item in enumerate(results):
        file.write(f"## Beispiel {i+1}\n{item}\n")

print(f"Results written to {output_file_path}")

Results written to output.txt
